Random Forests work on "wisdom of the crowd".
<br>Aggregating predictions of a group of predictors often gives better results.
<br>This is called "ensemble learning".

# Voting Classifiers

### Hard Voting Classifier
<br>Where we take majority-vote as aggregation.
<br>Even if each clf is a weak learner (i.e. slightly better than random guessing), the ensemble can be a strong learner (high accuracy)

This is possible due to law of large numbers: as we increase the number of learners (e.g. each giving 51% accuracy), the overall probability can be consistently above 50%.
<br>Assuming each learner makes independent, uncorrelated errors (which isn't the case cuz the training data used for them is same).


We can get diverse clfs by choosing different algos.

In [1]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
)

voting_clf.fit(X_tr, y_tr)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [2]:
# stores both original and cloned estimators
# clones are trained and available under estimators_ or named_estimators_
for name, clf in voting_clf.named_estimators_.items():
    print(f"{name} = {clf.score(X_ts, y_ts)}")

lr = 0.864
rf = 0.896
svc = 0.896


In [3]:
# majority voting
print(voting_clf.predict(X_ts[:1]))
print([clf.predict(X_ts[:1]) for clf in voting_clf.estimators_])

[1]
[array([1]), array([1]), array([0])]


In [4]:
voting_clf.score(X_ts, y_ts)

0.912

### Soft Voting Classifier
<br>If all clfs have predict_proba(), we can average over each clf's predicted prob and predict the one with highest prob.

Often better performance than Hard Voting, as it gives more weight to highly confident votes.

In [5]:
voting_clf.voting = "soft"
voting_clf.named_estimators["svc"].probability = True
voting_clf.fit(X_tr, y_tr)
voting_clf.score(X_ts, y_ts)

0.92

# Bagging and Pasting

Another way to get diverse clfs is to train same algo on different random subsets of training set.

When sampling w/ replacement, it's called "bagging" ("bootstrap aggregating").
<br>When sampling w/o replacement, it's called "pasting".

For classification, aggregation is statistical mode (most frequent prediction).
<br>For regression, it's average.

Aggregation reduces both bias and variance.

Bagging/Pasting are also popular because you can train all predictors and also make predictions in parallel, so they scale well.

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# bagging will automatically use soft voting if its clfs have predict_proba()
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            # n_jobs = number of CPU cores
                            # -1 means use all
                            max_samples=100, n_jobs=1, random_state=42)
bag_clf.fit(X_tr, y_tr)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=1, random_state=42)

Bagging is preferred over pasting due to extra diversity which means predictors are less correlated and variance is reduced.
<br>Due to diversity, the bias is slightly higher in Bagging than Pasting.
<br>Overall, it results in better models.

#### Out-of-bag (OOB) Evaluation
BaggingClassifier samples "m" instances w/ replacement (bootstrap=True).
<br>It is mathematically shown that only 63% of instances are sampled on average for each predictor.
<br>The remaining 37% are OOB instances.
<br>These 37% are different for each predictor.

In [7]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            oob_score=True, n_jobs=1, random_state=42)
bag_clf.fit(X_tr, y_tr)
bag_clf.oob_score_

0.896

In [8]:
# OOB score says it likely achieves 89.6% accuracy
# verify this
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_ts)
accuracy_score(y_ts, y_pred)

0.92

In [9]:
# probas for first 3 instances
# these are probas cuz base estimator has predict_proba()
bag_clf.oob_decision_function_[:3]

array([[0.32352941, 0.67647059],
       [0.3375    , 0.6625    ],
       [1.        , 0.        ]])

#### Random Patches and Random Subspaces
Sampling features helps if data is high-dim.

<br>Sampling both instances and features is called random patches.
<br>Sampling only features is called subspaces method.

bootstrap=False and max_samples=1.0 makes it not sample instances
<br>bootstrap_features=True and/or max_features < 1.0 makes it sample features

In [10]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            n_jobs=1, random_state=42,
                            bootstrap=False, bootstrap_features=True,
                            max_samples=1.0, max_features=0.75,
                            )
bag_clf.fit(X_tr, y_tr)

BaggingClassifier(bootstrap=False, bootstrap_features=True,
                  estimator=DecisionTreeClassifier(), max_features=0.75,
                  n_estimators=500, n_jobs=1, random_state=42)

# Random Forest

In [11]:
# rather than BaggingClassifier, we use RandomForestClassifier/Regressor which is optimized for DT
from sklearn.ensemble import RandomForestClassifier

# RF introduces extra randomness by sampling best feature from a random subset of features
# has params of both DT and BaggingClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,
                                 n_jobs=-1, random_state=42)
rnd_clf.fit(X_tr, y_tr)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [12]:
y_pred_rf = rnd_clf.predict(X_ts)

In [13]:
# equivalent BaggingClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_features='sqrt', max_leaf_nodes=16),
                  n_estimators=500, n_jobs=-1, random_state=42)

### Extra Trees

"Extremely Randomized Trees"
<br>Makes trees even more random by using random thresholds for each feature rather than best thresholds.
<br>Trades higher bias for lower variance
<br>Much faster to train cuz finding best threshold is time-consuming.

In [14]:
from sklearn.ensemble import ExtraTreesClassifier

extr = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16,
                            n_jobs=-1, random_state=42)
extr.fit(X_tr, y_tr)
extr.predict(X_ts)

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

### Feature importances

In [15]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris.data, iris.target)

for score, name in zip(rnd_clf.feature_importances_, iris.data.columns):
    print(round(score, 2), name)

0.11 sepal length (cm)
0.02 sepal width (cm)
0.44 petal length (cm)
0.42 petal width (cm)


In [16]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

rnd_clf_mnist = RandomForestClassifier(n_estimators=500, random_state=42,
                                       max_leaf_nodes=16, n_jobs=-1)
rnd_clf_mnist.fit(mnist.data, mnist.target)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [17]:
import matplotlib.pyplot as plt

# plt.gray()
# plt.matshow(mnist.images[0])
# plt.show()
for imp, name in zip(rnd_clf.feature_importances_, range(len(rnd_clf.feature_importances_))):
    print(imp, name)
len(rnd_clf.feature_importances_), mnist.data.shape

0.11249225099876375 0
0.02311928828251033 1
0.4410304643639577 2
0.4233579963547682 3


(4, (70000, 784))

# Boosting

### AdaBoost (adaptive boosting)

Algo increases relative weight of misclassified instances.
<br>So new clf are trained on errors of prev.
<br>This is done sequentially, so it can't be parallelized.

Weighted error rate of jth predictor:
<br>r_j = sum <sub>i=1 to m and y<sup>j</sup>(i) != y(i)</sub>(w(i))
<br>i.e. sum over incorrect predictions, with incorrect predictions having weight of w(i), set to 1/m initially

predictor's weight:
<br>alpha<sub>j</sub> = eta * log(1-r<sub>j</sub>) * r<sub>j</sub>
<br>More correct = higher weight
<br>Randomly guessing = close to 0
<br>Worse than random guess = -ve weight

Update rule:
for i = 1 to m:
<br>  w(i) = w(i) if correct prediction else, w(i) * e<sup>alpha<sub>j</sub></sup>
<br>Then normalize all instance weights (divide by sum<sub>from i=1 to m</sub>(w(i)))

Then retrain a new predictor on these updated weights, and repeat the process.
<br>i.e. compute new predictor's weight -> update instance weights -> train another predictor

For prediction,
<br>y<sup>(x)</sup> = argmax<sub>k</sub>(sum<sub>j=1 to N and y<sup>j</sup>(x) = k</sub>(alpha<sub>j</sub>))
<br>Basically, compute predictions using all predictors, weigh their predictions with predictor's weight and then use majority votes.

### SAMME
<br>Stagewise Additive Modeling using a Multiclass Exponential loss
<br>This is used by sklearn
<br>It's a multi-class version of AdaBoost.

If predictors have predict_proba(), then sklearn uses SAMME.R ("Real") which uses class probas rather than predictions and generally performs better.

In [18]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    # train Decision Stumps (max depth is 1)
    DecisionTreeClassifier(max_depth=1), n_estimators=30,
    learning_rate=0.5, random_state=42
)
ada_clf.fit(X_tr, y_tr)

c:\dev\miniconda\envs\homl-learn\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30, random_state=42)

### Gradient Boosting
Similar to AdaBoost, except it fits new predictors to the residual errors made by prev predictor, rather than tweaking instance weights.

using DT as base predictor is called Gradient Tree Boosting or Gradient Boosted Regression Trees

In [19]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3 * X[:, 0]**2 + 0.05 * np.random.randn(100)

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [20]:
# train the next one on the residual errors of previous
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [21]:
# again, train on residual errors
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [22]:
X_new = np.array([[-0.4], [0.], [0.5]])
# for prediction, we sum their predictions
sum(t.predict(X_new) for t in (tree_reg1, tree_reg2, tree_reg3))

array([0.49484029, 0.04021166, 0.75026781])

In [23]:
# Using sklearn
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3,
                                 learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

<br>In above, learning_rate scales the contribution of each tree.
<br>If we decrease the learning_rate, we'll need more trees to fit the training set but it'll generalize better.
<br>This regularization is called <b>shrinkage</b>

For optimal number of trees, we can use GridSearchCV or RandomizedSearchCV

Another simpler thing we can do is early stopping.
<br>Set `n_iter_no_change` and it'll stop if there's no progress for specified iterations.
<br>It's a bit more patient than early stopping (it tolerates for a few iterations with no progress before stopping)
<br>`tol` determines max performance improvement which counts.

When this param is set, `fit()` splits training set into training and val sets to evaluate performance each time it adds a new tree.
<br>The size of val set can be controlled by `validation_fraction`.

In [24]:
gbrt_best = GradientBoostingRegressor(max_depth=2, learning_rate=0.05, n_estimators=500,
                                      n_iter_no_change=10, random_state=42)
gbrt_best.fit(X, y)
gbrt_best.n_estimators_

92

### Stochastic Gradient Boosting
Specify `subsample` which is the fraction of instances to used for training each tree.
<br>This trades a higher bias for lower variance and speeds up training.

In [25]:
gbrt_best = GradientBoostingRegressor(max_depth=2, learning_rate=0.05, n_estimators=500,
                                      n_iter_no_change=10, random_state=42,
                                      subsample=0.25)
gbrt_best.fit(X, y)
gbrt_best.n_estimators_

71

## Histogram-Based Gradient Boosting
Optimized for large datasets

Works by binning the input features, replacing them with ints
- number of bins is controlled by `max_bins`, which is <= 255
- reduces number of possible thresholds that the algo needs to evaluate
- working w/ ints makes it possible to use better data structures
- binning causes a precision loss, which may act like a regularizer
  - depending on datasets, can lead to overfitting or underfitting
 
 Complexity is O(b * m) rather than O(n * m * lg(m))

Difference from GradientBoostingRegressor/Classifier:
- Early stopping is automatic if n > 10k
- no support for subsampling
- `n_estimators` is now called `max_iter`
- DT hyperparams are limited to `max_leaf_nodes`, `min_samples_leaf` and `max_depth`

HGB supports categorical features and missing values, so it simplifies preprocessing.

The categorical features must be represented as ints ranging from 0 to < max_bins
<br>Can do this via OrdinalEncoder

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

def load_housing_data():
    from pathlib import Path
    import pandas as pd
    import tarfile
    import urllib.request

    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()

hgb_reg = make_pipeline(
    make_column_transformer(
        (OrdinalEncoder(), ['ocean_proximity']),
        remainder="passthrough"
        ),
        HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
)
hgb_reg.fit(housing.drop(['median_house_value'], axis=1), housing.median_house_value)

c:\dev\miniconda\envs\homl-learn\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['ocean_proximity'])])),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor(categorical_features=[0],
                                               random_state=42))])

### Other GB Implementations
XGBoost, CatBoost, LightGBM, TensorFlow Random Forests


# Stacking
Short for "Stacked generalization".
<br>Idea: Train a model for aggregation instead of trivial functions like hard voting

The final predictor ("blender" or "meta-learner") takes predictions and makes final prediction

To train the blender, need a blending training set.
<br>This can be made using cross_val_predict() of every predictor to get out-of-sample predictions.

Can train multiple blenders and add another blender on top of that.
<br>But this costs more training time and system complexity.

In [34]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    # uses LogisticRegression as default
    # StackingRegressor would use RidgeCV
    final_estimator=RandomForestClassifier(random_state=42),
    cv=5
)

stacking_clf.fit(X_tr, y_tr)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=42))

In [35]:
y_pred_ts = stacking_clf.predict(X_ts)
accuracy_score(y_ts, y_pred_ts)

0.912